In [ ]:
import pandas as pd
import glob
import os
import unicodedata
from thefuzz import fuzz
from thefuzz import process

In [ ]:
#Definição do método que transforma, e concatena, arquivos CSVs em um único Dataset, 
# com a opção de acrescentar a coluna SEASON a partir do nome do arquivo
def fromcsv_todf(path,fl_folder=False, get_season_from_filename = False):
       
    if get_season_from_filename == True:
        if fl_folder == True:
            d_list = []
            dfs =[]

            for file in os.listdir(path):
                d = os.path.join(path, file)
                d_list.append(d)

            for filepath in d_list:
                filenames = glob.glob(filepath + "/*.csv")
                for csv in filenames: 
                    df1 = pd.read_csv(csv)
                    df1["nr_season"]=int(csv[-8:-4])
                    dfs.append(df1)

            csv_unique = pd.concat(dfs, ignore_index=True)
            return csv_unique

        else:
            dfs =[]
            df1 = pd.read_csv(csv)
            df1["nr_season"]=int(path[-8:-4])
            dfs.append(df1)
            csv_unique = pd.concat(dfs, ignore_index=True)
            return csv_unique
          
        
    else:
        if fl_folder == True:
            d_list = []
            dfs =[]

            for file in os.listdir(path):
                d = os.path.join(path, file)
                d_list.append(d)

            for filepath in d_list:
                filenames = glob.glob(filepath + "/*.csv")
                for csv in filenames:
                    dfs.append(pd.read_csv(csv))

            csv_unique = pd.concat(dfs, ignore_index=True)
            return csv_unique

        else:
            dfs =[]
            dfs.append(pd.read_csv(path))
            csv_unique = pd.concat(dfs, ignore_index=True)
            return csv_unique

In [ ]:
#Definição do método que cria uma lista de clubes, removendo suas duplicidades, a partir de um Dataset
def create_club_list_from_df(csv_unique, club_column_name):
    equipe_format = []
    for equipe in csv_unique[club_column_name]:
        try:
            equipe_format.append(unicodedata.normalize('NFD',equipe.split(" - ")[0]).encode('ascii', 'ignore').decode("utf-8"))
        except:
            equipe_format.append(unicodedata.normalize('NFD',str(equipe)).encode('ascii', 'ignore').decode("utf-8"))

    equipe_format = list(dict.fromkeys(equipe_format))

    df_tm = pd.DataFrame.from_dict({club_column_name : equipe_format})
            
    club_list_tm = list(df_tm[club_column_name])
    club_list_tm = [x for x in club_list_tm if str(x) != 'nan']
    
    return club_list_tm

In [ ]:
#definição do método que fará o match entre os nomes (jogadores e clubes), para posterior merge dos datasets
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    fl_match = 0
    for x in list_names:
        score = fuzz.ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = str(x)
            max_score = score
            fl_match = 1
    return max_name, max_score, fl_match


In [ ]:
#Geração do dataset de de-paras de nomes das equipes, entre TransferMarket e API FOOTBALL
def get_depara_df(club_list_stats, club_list_tm):
    
    club_list_stats_df = []
    club_list_tm_df = []
    match_score_list_df = []
    
    for x in club_list_stats:
        match = match_names(x, club_list_tm, -1)
        club_list_stats_df.append(str(x)) 
        club_list_tm_df.append(str(match[0]))
        match_score_list_df.append(match[1])

    df_depara_clubs = pd.DataFrame.from_dict({"Club_name_stats" : club_list_stats_df, "Club_name_tm" : club_list_tm_df, 
                                              "Match_score" : match_score_list_df})
    
    return df_depara_clubs

In [ ]:
#Método para extração do número da temporada do Dataset do TransferMarket, a partir do campo "Nome da Equipe"
def getSeasonFromTmData(df):
    
    club_name_list =[]
    season_list = []
    for index, row in df.iterrows():
        try:
            club = unicodedata.normalize('NFD',str(row['Equipe'].split(' - Season ')[0])).encode('ascii', 'ignore').decode("utf-8")
            season = str(row['Equipe'].split(' - Season ')[1]).lstrip().rstrip()
            if season[2] == r'/':
                season = str(20) + str(season[-2:])
            season_list.append(int(season))
            club_name_list.append(club)
        except:
            club_name_list.append(unicodedata.normalize('NFD',str(row['Equipe'])).encode('ascii', 'ignore').decode("utf-8"))
            season_list.append(2022)
    
    df["nr_season"] = season_list
    df["club_names_tm"] = club_name_list
    
    return df

In [ ]:
##Inserir path dos dados capturados do TransferMarket
path = r'C:\Users\Public\TransferMarketData' 

##Inserir path dos dados capturados da API FOOTBALL
path_2 = r'C:\Users\User\Downloads\Serie_A_League-20220906T230501Z-001' 

##Inserir path do dataset de de-paras dos nomes dos clubes entre TransferMarket e API FOOTBALL
path_3 = r'C:\Users\User\Downloads\de_para_nome_clubes.csv' 

##Inserir nome da coluna que recebe o nome da Equipe na qual oo jogador atuou, do dataset do TransferMarket
column_name = 'Equipe'

##Inserir nome da coluna que recebe o nome da Equipe na qual oo jogador atuou, do dataset do API FOOTBALL
column_name_2 = "statistics_team_name"

In [ ]:

#Leitura do Dataset do TransferMarket e tratamento do nome do clube Atletico Paranaense
df_tm = getSeasonFromTmData(fromcsv_todf(path,True))
for index in df_tm.index:
    if df_tm.loc[index, "club_names_tm"] == "Atletico Paranaense":
        df_tm.loc[index, "club_names_tm"] = "Club Athletico Paranaense"

#Leitura do Dataset do API FOOTBALL, criação dos campos Full Player Name, para melhoriar a performance do match dos Datasets
df_stats = fromcsv_todf(path_2,True,True)
for index in df_stats.index:
    df_stats.loc[index, "full_player_name"] = str(df_stats.loc[index, "player_firstname"]) + " " + str(df_stats.loc[index, "player_lastname"])
    df_stats.loc[index, "full_player_name"] = unicodedata.normalize('NFD',str(df_stats.loc[index, "full_player_name"])).encode('ascii', 'ignore').decode("utf-8")

#Leitura do Dataset de de-paras de nomes dos clubes
df_depara = fromcsv_todf(path_3)

#config para display de todas as colunas dos Datasets
pd.set_option('display.max_columns', None)

#Merge entre os Datasets API FOOTBALL e DE-PARAs, para normalizar o nome dos clubes
df_stats = df_stats.merge(df_depara, left_on="statistics_team_name", right_on="club_names_stats", how="left")

#Geração de Dataset looping, que será base do processo de merge entre Transfermarket e API FOOTBALL
df_stats_loop= df_stats[["club_names_tm", "nr_season", "full_player_name"]]

In [ ]:
#Execução do loop qque fará o match entre o nome dos jogadores de ambos os datasets TransferMarket e API FOOTBALL
#O objetivo é enriquecer o dataset API FOOTBALL com os valores de mercado dos jogadores a partir do dataset TransferMarket
#Para refinar e aumentar as chances de match, foram realizados loop que buscam o match por jogador, temporada e clube atuante
#Por fim, com um match-score rating pré-acordado em 75 (em uma escala de -100 a 100), foi acrescido o valor de mercado do jogador

for index in df_tm.index:

    season_filter = int(df_tm.loc[index, "nr_season"])
    club_name_filter = str(df_tm.loc[index, "club_names_tm"])
    player_name_tm = str(df_tm.loc[index, "Jogador"])
    player_value_tm = str(df_tm.loc[index, "Valor_de_Mercado"])
    
    df_stats_filtered = df_stats_loop[(df_stats_loop["nr_season"] == season_filter) & (df_stats_loop["club_names_tm"] == club_name_filter)]
    players_stats_name_list = df_stats_filtered[["full_player_name"]].values.tolist()
    
    match = match_names(player_name_tm, players_stats_name_list, 75)
    match_player_name = match[0][2:-2]
    match_score = match[1]
    match_flag = match[2]
    
    if (match_flag == 1):
            print(str(season_filter) + " - " + str(club_name_filter) + " - " + str(player_name_tm)  + " - " + match_player_name + " - " + str(match_score) + " - " + str(match_flag))
            
            update_condition = (df_stats["nr_season"] == season_filter) & (df_stats["club_names_tm"] == club_name_filter) & (df_stats["full_player_name"] == match_player_name)
            index_get = df_stats.index
            index_update = index_get[update_condition]
            
            df_stats.loc[index_update[0], "tm_player_value"] = player_value_tm
            df_stats.loc[index_update[0], "name_match_score"] = match_score
            df_stats.loc[index_update[0], "name_match_name"] = player_name_tm
            df_tm.loc[index, "match_index"] = int(index)
            

2016 - America Futebol Clube (MG) - Joao Ricardo Riedi - Joao Ricardo Riedi - 90 - 1
2016 - America Futebol Clube (MG) - Fernando Leal Fonseca - Fernando Leal Fonseca - 91 - 1
2016 - America Futebol Clube (MG) - Joriwinnyson Santos dos Anjos Rodrigues - Joriwinnyson Santos dos Anjos Rodrigues - 95 - 1
2016 - America Futebol Clube (MG) - Artur Jesus Vieira - Artur Jesus Vieira - 90 - 1
2016 - America Futebol Clube (MG) - Eder Lima dos Santos - Eder Lima dos Santos - 91 - 1
2016 - America Futebol Clube (MG) - Alison Wagner Lira Ferreira - Alison Wagner Lira Ferreira - 93 - 1
2016 - America Futebol Clube (MG) - Sueliton Florencio Nogueira - Sueliton Florencio Nogueira - 93 - 1
2016 - America Futebol Clube (MG) - Carlos Alexandre Cardoso - Carlos Alexandre Cardoso - 92 - 1
2016 - America Futebol Clube (MG) - Adalberto dos Santos Rodrigues - Adalberto dos Santos Rodrigues - 94 - 1
2016 - America Futebol Clube (MG) - Roger Duarte de Oliveira - Roger Duarte de Oliveira - 92 - 1
2016 - America

In [ ]:
#Por fim, donload do dataset completo, para posterior EDA
local_path = r"C:\Users\User\Downloads\data_stats_completo.csv" ##Inserir pasta local para download do Dataset completo
df_stats.to_csv(local_path, index=False)